In [1]:
from typing import Dict

# STANDARD_TYPES
INT = "INT"
FLOAT = "FLOAT"
NUMERIC = "NUMERIC"
TEXT = "TEXT"
TIMESTAMP = "TIMESTAMP"
DATETIME_PANDAS = "DATETIME64[NS]"

# STANDARD_DATA_SOURCES
PANDAS = "PANDAS"

DIALECT_MAPPING: Dict[str, Dict[str, str]] = dict(
    PANDAS=dict(
        OBJECT=TEXT,
        DATETIME_PANDAS=TIMESTAMP,
        INT64=TEXT,
        FLOAT64=TEXT,
    )
)

In [2]:
import os
from typing import List, Optional
import pandas as pd
from pandas import DataFrame

def get_tables_by_prefix_separator(data_directory, prefix_separator: str = "_") -> List[str]:
    potential_tables = [pt.split(prefix_separator)[0] for pt in os.listdir("/dataset")]
    tables = set(potential_tables)
    return list(tables)

def get_csv_data_files_for_table(dataset_directory: str, table: str, prefix_separator: str) -> List[str]:
    data_files_glob = f"{table}{prefix_separator}*.csv"
    data_files = glob.glob(os.path.join(dataset_directory, data_files_glob))
    return data_files

class DataFrameTools(object):
    def __init__(self):
        pass

    def get_typed_dataframe_from_file(self, data_file: str, date_columns: List[str]):
        df = pd.read_csv(data_file, parse_dates=date_columns)
        return df

    def get_dtypes_dict_from_typed_dataframe(self, typed_df: DataFrame):
        dtypes_dict = typed_df.dtypes.to_dict()
        for key, value in dtypes_dict.items():
            dtypes_dict[key] = str(value).upper()
        return dtypes_dict


In [3]:
import codecs
import glob
from io import StringIO
from jinja2 import Environment, FileSystemLoader
import os
import psycopg2
import sys
import traceback
from typing import Any, Dict

DEFAULT_USER = "postgres"
DEFAULT_PASSWORD = "password"
DEFAULT_DATABASE = "take_home"
DEFAULT_SCHEMA = "eric_meadows"
DEFAULT_HOST = "database"

DEFAULT_POTENTIAL_NULL_COLUMNS = ["event_ts"]



TEMPLATE_DIR = os.path.join(os.path.dirname("./jinja/"), "templates")
TEMPLATE_LOADER = FileSystemLoader(searchpath=TEMPLATE_DIR)
TEMPLATE_ENV = Environment(loader=TEMPLATE_LOADER)


TABLE_DATE_COLUMNS: Dict[str, List[str]] = dict(
    user=["event_ts",],
    marketing=["event_ts",]
)

DROP_AND_CREATE_TABLE_WITH_SCHEMA = "drop_and_create_table_with_schema.jinja2"
CREATE_SCHEMA_IF_NOT_EXISTS = "create_schema_if_not_exists.jinja2"

class DatabaseTools(object):
    def __init__(self, *args, **kwargs):
        pass

    def load_jinja_query_template(
        self,
        template_file: str,
        params: Dict[str, Any]
    ) -> str:
        template = TEMPLATE_ENV.get_template(template_file)
        rendered_query = template.render(params)
        return rendered_query
    

class PostgresTools(DatabaseTools):
    def __init__(
        self,
        host: str = DEFAULT_HOST,
        database: str = DEFAULT_DATABASE,
        user: str = DEFAULT_USER,
        password: str = DEFAULT_PASSWORD,
        *args,
        **kwargs
    ):
        super(PostgresTools, self).__init__(*args, **kwargs)
        self.host = host
        self.database = database
        self.user = user
        self.password = password
        self.conn = psycopg2.connect(
            host=self.host,
            dbname=self.database,
            user=self.user,
            password=self.password
        )
        self.dataframe_tools = DataFrameTools()
    
    def _ensure_schema_present(
        self,
        schema,
    ):
        query_params = dict(
            schema=schema
        ) 
        query = self.load_jinja_query_template(
            CREATE_SCHEMA_IF_NOT_EXISTS,
            query_params
        )
        self._run_query(query)

    def _drop_and_create_table_sql(
        self,
        database: str,
        table: str,
        dialect: str,
        schema_map: Dict[str, str],
    ):
        dialect_map = DIALECT_MAPPING[dialect]
        postgres_table_name = f"{database}.{table}"

        query_params = dict(
            postgres_table_name=postgres_table_name,
            mapping=dialect_map,
            schema_map=schema_map,
        )
        query_sql = self.load_jinja_query_template(
            DROP_AND_CREATE_TABLE_WITH_SCHEMA,
            query_params
        )
        return query_sql

    def _drop_and_create_table(
        self,
        schema: str,
        table: str,
        dialect: str,
        schema_map: Dict[str, str],
    ):
        query = self._drop_and_create_table_sql(
            schema,
            table,
            dialect,
            schema_map
        )
        self._run_query(query)
    
    def _get_file_buffer_without_null_bytes(
        self,
        data_file: str,
        potential_null_columns: List[str] = DEFAULT_POTENTIAL_NULL_COLUMNS
    ):
        writeable_csv_buffer = StringIO()
        pd.read_csv(data_file)\
          .dropna(subset=potential_null_columns)\
          .to_csv(writeable_csv_buffer, index=False, header=False)
        writeable_csv_buffer.seek(0)
        return writeable_csv_buffer
    
    def _load_file_into_table(
        self,
        schema: str,
        table: str,
        data_file: str,
        separator=","
    ):
        cursor = self.conn.cursor()
        writeable_csv_buffer = self._get_file_buffer_without_null_bytes(data_file)
        try:
            cursor.copy_from(writeable_csv_buffer, f"{schema}.{table}", sep=separator, null="")
            self.conn.commit()
        except Exception as err:
            traceback.print_exc(file=sys.stdout)
            raise err
        finally:
            writeable_csv_buffer.close()
            cursor.close()
    
    def _run_query(self, query):
        
        cursor = self.conn.cursor()
        cursor.execute(query)
        self.conn.commit()
        cursor.close()
    
    def load_files_into_database(
        self,
        schema: str,
        table: str,
        data_files: List[str],
        dialect: str = PANDAS,
    ):
        first_data_file = data_files[0]
        df = self.dataframe_tools.get_typed_dataframe_from_file(first_data_file, [])
        schema_map = self.dataframe_tools.get_dtypes_dict_from_typed_dataframe(df)
        
        self._ensure_schema_present(schema)

        self._drop_and_create_table(
            schema,
            table,
            dialect,
            schema_map,
        )
        for data_file in data_files:
            self._load_file_into_table(schema, table, data_file)


In [4]:
from typing import Dict

TABLE_DATE_COLUMNS: Dict[str, List[str]] = dict(
    user=["event_ts",],
    marketing=["event_ts",]
)
DATASET_DIRECTORY = "/dataset"
PREFIX_SEPARATOR = "_"


def main():
    postgres_tools = PostgresTools()

    required_tables = get_tables_by_prefix_separator(DATASET_DIRECTORY, PREFIX_SEPARATOR)
    for table in required_tables:
        print(f"Loading data for table:  {table}")
        data_files = get_csv_data_files_for_table(DATASET_DIRECTORY, table, PREFIX_SEPARATOR)
        result = postgres_tools.load_files_into_database(DEFAULT_SCHEMA, table, data_files)

if __name__ == "__main__":
    main()


Loading data for table:  marketing
Loading data for table:  user


# Investigation

In [5]:
from sqlalchemy import create_engine
engine = create_engine(f'postgresql://{DEFAULT_USER}:{DEFAULT_PASSWORD}@{DEFAULT_HOST}:5432/{DEFAULT_DATABASE}')

USERS_TABLE = "user"
MARKETING_TABLE = "marketing"

users_df = pd.read_sql_query(
    f"select * from {DEFAULT_SCHEMA}.{USERS_TABLE}",
    con=engine,
    parse_dates=["event_ts"])
marketing_df = pd.read_sql_query(
    f"select * from {DEFAULT_SCHEMA}.{MARKETING_TABLE}",
    con=engine,
    parse_dates=["event_ts"])
marketing_df.length = marketing_df.length.astype("float")


In [6]:
print("[Python]")

# Question 1
unique_user_count = len(users_df.user_id.unique())
print(f"1.  Unique users:  {unique_user_count}")

# Question 2
marketing_providers = ", ".join(list(filter(None, marketing_df.provider.unique())) )
print(f"2.  Marketing providers:  {marketing_providers}")

# Question 3
change_frequency = users_df.property.value_counts()
most_changed_attribute_name = change_frequency.idxmax().title()
most_changed_attribute_count = change_frequency.max()
print(f"3.  Most-changed Attribute - Name:  {most_changed_attribute_name}, Count:  {most_changed_attribute_count}")

# Question 4
DATE = "2019-07-03"
PROVIDER = "Snapchat"
users_show_ad_on_provider_on_given_date = marketing_df\
    .loc[marketing_df.event_ts.dt.strftime("%Y-%m-%d") == DATE]\
    .loc[marketing_df.provider == PROVIDER]\
    .shape[0]
print(f"4.  Users shown an ad on {PROVIDER} on {DATE}:  {users_show_ad_on_provider_on_given_date}")

# Question 5
POLITICAL_AFFILIATION = "moderate"
phone_id_and_political_affiliation = users_df.loc[
    users_df.property == "politics",
    ["phone_id", "value"]]\
    .dropna(subset=["value"])\
    .rename(columns={"value": "political_affiliation"})
merged_df = pd.merge(
    marketing_df,
    phone_id_and_political_affiliation,
    how="inner",
    left_on="phone_id",
    right_on="phone_id")
most_shown_ad_to_specific_political_affliation = merged_df\
    .loc[merged_df.political_affiliation.str.lower() == POLITICAL_AFFILIATION]\
    .ad_id\
    .value_counts()\
    .idxmax()
print(f"5.  Ad shown most to {POLITICAL_AFFILIATION}s:  {most_shown_ad_to_specific_political_affliation}")

# Work for Question 6
question_6_data = marketing_df.groupby(by=["ad_id"])\
    .agg({"length": ["mean", "std"], "phone_id": "nunique"})\
    .sort_values([
        ('phone_id', "nunique"),
        ("length", "mean")],
    ascending=False)

[Python]
1.  Unique users:  2904
2.  Marketing providers:  Facebook, Instagram, Spotify, Snapchat, Inst
3.  Most-changed Attribute - Name:  Drinking, Count:  1473
4.  Users shown an ad on Snapchat on 2019-07-03:  261
5.  Ad shown most to moderates:  4


In [7]:
print("[SQL]")

# Question 1
distinct_users = pd.read_sql_query(
    f"""
SELECT
  COUNT(DISTINCT(user_id))
FROM
  {DEFAULT_SCHEMA}.{USERS_TABLE}""",
    con=engine)
number_of_distinct_users = distinct_users.iloc[0].loc["count"]
print(f"1.  Unique users:  {number_of_distinct_users}")

# Question 2
marketing_providers_list = pd.read_sql_query(
    f"""
SELECT
  DISTINCT(provider)
FROM
  {DEFAULT_SCHEMA}.{MARKETING_TABLE}
ORDER BY provider""",
    con=engine).values.flatten().tolist()
marketing_providers = ", ".join(marketing_providers_list)
print(f"2.  Marketing providers:  {marketing_providers}")

# Question 3
most_changed_property = pd.read_sql_query(
    f"""
SELECT
  property, COUNT(*) AS _count
FROM
  {DEFAULT_SCHEMA}.{USERS_TABLE}
GROUP BY
  property
ORDER BY
  _count DESC
LIMIT
  1""",
    con=engine).loc[0]
print(f"3.  Most-changed Attribute - Name:  {most_changed_property.property}, Count:  {most_changed_property._count}")

# Question 4
DATE = "2019-07-03"
PROVIDER = "Snapchat"
users_show_ad_on_provider_on_given_date = pd.read_sql_query(
    f"""
SELECT
  COUNT(1) AS _count
FROM
  (
    SELECT
      *, event_ts::TIMESTAMP AS event_timestamp
    FROM
      {DEFAULT_SCHEMA}.{MARKETING_TABLE}
  ) AS normalized_users
WHERE
  event_timestamp BETWEEN DATE '{DATE}' AND DATE '{DATE}' + INTERVAL '1 day' AND
  provider = '{PROVIDER}'
  ;
""",
    con=engine).loc[0, "_count"]
print(f"4.  Users shown an ad on {PROVIDER} on {DATE}:  {users_show_ad_on_provider_on_given_date}")

# Question 5
POLITICS = "politics"
POLITICAL_AFFILIATION = "moderate"
most_shown_ad_to_specific_political_affliation = pd.read_sql_query(
    f"""
SELECT
  ad_id
FROM
  {DEFAULT_SCHEMA}.{MARKETING_TABLE}
WHERE
  phone_id IN (
  SELECT
    phone_id
  FROM
    (
      SELECT
        phone_id, property, LOWER(value) AS lower_value
      FROM
        {DEFAULT_SCHEMA}.{USERS_TABLE}
    ) AS standardized_users
  WHERE
    property = '{POLITICS}' AND
    lower_value = '{POLITICAL_AFFILIATION}'
  )
GROUP BY
  ad_id
ORDER BY
  COUNT(1) DESC
LIMIT
  1
""",
    con=engine).loc[0, "ad_id"]
print(f"5.  Ad shown most to {POLITICAL_AFFILIATION}s:  {most_shown_ad_to_specific_political_affliation}")

# Question 6
most_successful_ads = pd.read_sql_query(
    f"""
SELECT
  _user_groups AS "user_groups",
  ad_id,
  _distinct_users AS "distinct_users",
  _mean AS "view_time_mean",
  _percent_1 AS "view_time_percentile_1",
  _percent_99 AS "view_time_percentile_99"
FROM
  (
    SELECT
      *, MAX(_percent_1) OVER (PARTITION BY _user_groups) AS _max_percent_1
    FROM
      (
        SELECT
          ad_id,
          _distinct_users,
          _mean,
          _mean - 2 * _stddev AS _percent_1,
          _mean + 2 * _stddev AS _percent_99,
          CASE
            WHEN _distinct_users BETWEEN 0   AND 100 THEN '1'
            WHEN _distinct_users BETWEEN 100 AND 159 THEN '2'
            WHEN _distinct_users BETWEEN 160 AND 299 THEN '3'
            WHEN _distinct_users BETWEEN 300 AND 499 THEN '4'
            ELSE '5'
          END AS _user_groups
        FROM
          (
            SELECT
              ad_id, COUNT(phone_id) AS _distinct_users, AVG(length::FLOAT) AS _mean, STDDEV(length::FLOAT) AS _stddev
            FROM
              {DEFAULT_SCHEMA}.{MARKETING_TABLE}
            GROUP BY
              ad_id
          ) AS stats_marketing
        ORDER BY
          _distinct_users DESC
      ) AS stats_marketing_metrics
  ) AS stats_marketing_metrics_with_max
WHERE
  _percent_1 = _max_percent_1
""",
    con=engine)
most_successful_ads

[SQL]
1.  Unique users:  2903
2.  Marketing providers:  Facebook, Inst, Instagram, Snapchat, Spotify
3.  Most-changed Attribute - Name:  drinking, Count:  1473
4.  Users shown an ad on Snapchat on 2019-07-03:  261
5.  Ad shown most to moderates:  4


,user_groups,ad_id,distinct_users,view_time_mean,view_time_percentile_1,view_time_percentile_99
0,1,20,97,1237.164948,-133.553649,2607.883546
1,2,17,145,1162.662069,-76.760830,2402.084968
2,3,12,173,1232.901734,-186.057263,2651.860731
3,4,5,379,1129.005277,-216.018867,2474.029421
4,5,0,673,1251.282318,-135.784754,2638.349390


In [8]:
print("Table for Markdown in ANSWERS.md:\n")
print(most_successful_ads.to_csv(index=False))

Table for Markdown in ANSWERS.md:

user_groups,ad_id,distinct_users,view_time_mean,view_time_percentile_1,view_time_percentile_99
1,20,97,1237.16494845361,-133.553648986325,2607.88354589354
2,17,145,1162.66206896552,-76.7608303596678,2402.0849682907
3,12,173,1232.90173410405,-186.057263037625,2651.86073124572
4,5,379,1129.00527704485,-216.018866596538,2474.02942068625
5,0,673,1251.2823179792,-135.78475381805,2638.34938977644



# Python
## Examining the data above, it appears clear that there are 4 ad groups based upon the number of users an ad was shown to
## Because of this, there appears to be a winner for each group, and then we will select another
### Group 1:  >500 users shown
Winner:  ad_id = 0
Reason:  mean - 2 std. dev > every other in the group
### Group 2:  300-499 users shown
Winner:  ad_id = 5
Reason:  mean - 2 std. dev > every other in the group
### Group 4:  160-299 users shown
Winner:  ad_id = 12
Reason:  mean - 2 std. dev > every other in the group
### Group 3:  100-159 users shown
Winner:  ad_id = 14
Reason:  mean - 2 std. dev > every other in the group
### Group 4:  <100 users shown
Winner:  ad_id = 20
Reason:  mean - 2 std. dev > every other in the group